# Notebook - Gold Analysis - Part 1
Notebook utilizado para realizar as análises dos dados da camada Gold - Parte 1 do Case.

No caso:
- Geração das métricas Taxa de rentenção, Número médio de pedidos, Ticket médio e Receita média
- Execução dos testes T (indicadores de média) e Z (indicadores de proporção) para análise de impacto significativo

In [0]:
# Import das bibliotecas
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest

In [0]:
%sql
-- Geração das métricas Taxa de rentenção, Número médio de pedidos, Ticket médio e Receita média
select
  is_target
  , sum(is_retained)/count(*) as retention_rate
  , avg(n_order) as avg_n_order
  , sum(sum_amount)/sum(n_order) as avg_amount_per_order
  , sum(sum_amount)/count(*) as avg_sum_amount
  , sum(sum_amount) as total_sum_amount
  , count(*) as n_consumers
  , case when is_target = 'target' then n_consumers*20 else 0 end as campaign_cost
  , total_sum_amount*0.175 as profit
  , case when is_target = 'target' then (profit - campaign_cost) / campaign_cost else 0 end as roi
from gold.consumer_metrics
group by is_target

In [0]:
# Carregando os dados da camada Gold e transformando em DataFrames Pandas
df_consumer_metrics = spark.sql("""select * from gold.consumer_metrics""")
df_order_metrics = spark.sql("""select * from gold.order_metrics""")

df_consumer_metrics_pandas = df_consumer_metrics.toPandas()
df_order_metrics_pandas = df_order_metrics.toPandas()

In [0]:
# Taxa de retenção

count = [
    len(df_consumer_metrics_pandas[(df_consumer_metrics_pandas["is_target"] == "target") & (df_consumer_metrics_pandas["is_retained"] == 1)]), 
    len(df_consumer_metrics_pandas[(df_consumer_metrics_pandas["is_target"] == "control") & (df_consumer_metrics_pandas["is_retained"] == 1)]),
]
nobs = [
    len(df_consumer_metrics_pandas[df_consumer_metrics_pandas["is_target"] == "target"]), 
    len(df_consumer_metrics_pandas[df_consumer_metrics_pandas["is_target"] == "control"])
]

# Teste Z
stat, p_val = proportions_ztest(count, nobs)
print(f"P-valor (Taxa de retenção): {p_val:.4f}")

In [0]:
# Número médio de pedidos

target_n_order = df_consumer_metrics_pandas[df_consumer_metrics_pandas["is_target"] == "target"]["n_order"]
control_n_order = df_consumer_metrics_pandas[df_consumer_metrics_pandas["is_target"] == "control"]["n_order"]

# Teste T
t_stat, p_val = ttest_ind(target_n_order, control_n_order, equal_var=False)
print(f"P-valor (Número médio de pedidos): {p_val:4f}")

In [0]:
# Ticket médio

target_total_amount = df_order_metrics_pandas[df_order_metrics_pandas["is_target"] == "target"]["order_total_amount"]
control_total_amount = df_order_metrics_pandas[df_order_metrics_pandas["is_target"] == "control"]["order_total_amount"]

# Teste T
t_stat, p_val = ttest_ind(target_total_amount, control_total_amount, equal_var=False)
print(f"P-valor (Ticket médio): {p_val:4f}")

In [0]:
# Receita Média

target_sum_amount = df_consumer_metrics_pandas[df_consumer_metrics_pandas["is_target"] == "target"]["sum_amount"]
control_sum_amount = df_consumer_metrics_pandas[df_consumer_metrics_pandas["is_target"] == "control"]["sum_amount"]

# Teste T
t_stat, p_val = ttest_ind(target_sum_amount, control_sum_amount, equal_var=False)
print(f"P-valor (Receita Média): {p_val:4f}")